## Prepare the dataset 

Prepare the dataset so that it can be used to label the data.

- Posts only posted in 2024 through 2025 
- Combine the title and text 
- Remove spaces in combine_text
- Add in identifier column


In [1]:
%%capture
pip install -r ../requirements.txt

In [2]:
# Install the required packages
import pandas as pd
from datetime import datetime 

# NLP
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Need this set to None otherwise text columns will truncate!
pd.set_option('display.max_colwidth', None) 

In [3]:
import sys

sys.path.append("../scripts/")

In [4]:
from access_data import authenticate_google_drive, grab_google_drive_folder_data, save_google_drive_data

drive = authenticate_google_drive('credentials/google_drive_client_secret.json')
df = grab_google_drive_folder_data(drive=drive,credential_file="credentials/google_drive_folder_id.json",filename="reddit_data.csv")

Successfully loaded 'reddit_data.csv' into a DataFrame!


In [5]:
# Convert the 'timestamp' column from object (string) to datetime
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')

# Filter data between 2023 and 2025
filtered_df = df[(df['created_at'].dt.year >= 2020) & (df['created_at'].dt.year <= 2025)]


In [6]:
filtered_df.isna().sum()

submission_id             0
subreddit_topic           0
search_query              0
title                     0
text                    323
url                       0
score                     0
num_comments              0
username                  0
created_at                0
data_pull_date            0
days_since_post_date      0
dtype: int64

In [7]:
# replace NA to ''
filtered_df['text'] = filtered_df['text'].replace(pd.NA,"")


In [8]:
filtered_df.isna().sum()

submission_id           0
subreddit_topic         0
search_query            0
title                   0
text                    0
url                     0
score                   0
num_comments            0
username                0
created_at              0
data_pull_date          0
days_since_post_date    0
dtype: int64

In [9]:
## Combine text and remove spacing 
filtered_df['combine_text'] = filtered_df['title']+". "+ filtered_df['text']
filtered_df['combine_text'] = filtered_df['combine_text'].str.strip().str.replace(r'\s+', ' ', regex=True)


In [10]:
# We only want to analyze text data not photos
def flag_picture_posts(url): 
    if url.endswith(('.jpg', ".jpeg", ".png",'.gif', '.gifv', '.webp')):
        return True
    return False

filtered_df['flag_picture_posts']  = filtered_df['url'].apply(lambda x: flag_picture_posts(x))

In [11]:
# How many are images
filtered_df["flag_picture_posts"].value_counts()


False    4161
True      348
Name: flag_picture_posts, dtype: int64

In [12]:
filtered_df = filtered_df[filtered_df["flag_picture_posts"] == False].reset_index(drop=True) 


In [13]:
# Some duplicate posts since the same post may mention two brands 
print('Number of unique submission ids: ', filtered_df['submission_id'].nunique())
print('Number of total submission ids: ', filtered_df['submission_id'].count())

# Mini investigation that shows the posts have two different brands 
# submission_id_dup = filtered_df['submission_id'].value_counts().head()
# submission_id_example = submission_id_dup.index[0]
# filtered_df[filtered_df['submission_id'] == submission_id_example].head()

Number of unique submission ids:  4068
Number of total submission ids:  4161


In [14]:
# Save the data in the Google Drive location
save_google_drive_data(drive=drive, 
                       dataframe =filtered_df, 
                       filename="reddit_filtered_data.csv")

File 'reddit_filtered_data.csv' uploaded successfully to folder 1Ktcv4eaR7kH0teyGuLph4LSYWxI1qkIS!
